<a href="https://colab.research.google.com/github/mphfernando/Bitcoin-Price-Prediction/blob/main/Random_forest_Bitcoin_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance pandas matplotlib scikit-learn

In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Download daily OHLC data for BTC-USD from 2017 to today
btc_data = yf.download('BTC-USD', start='2019-01-01', end='2025-08-10', interval='1d')

# Display top rows
print(btc_data.head())

/tmp/ipython-input-1190875175.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc_data = yf.download('BTC-USD', start='2019-01-01', end='2025-08-10', interval='1d')
[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open      Volume
Ticker          BTC-USD      BTC-USD      BTC-USD      BTC-USD     BTC-USD
Date                                                                      
2019-01-01  3843.520020  3850.913818  3707.231201  3746.713379  4324200990
2019-01-02  3943.409424  3947.981201  3817.409424  3849.216309  5244856836
2019-01-03  3836.741211  3935.685059  3826.222900  3931.048584  4530215219
2019-01-04  3857.717529  3865.934570  3783.853760  3832.040039  4847965467
2019-01-05  3845.194580  3904.903076  3836.900146  3851.973877  5137609824


In [4]:
# Check for any missing values in the DataFrame
print(btc_data.isnull().sum())

Price   Ticker 
Close   BTC-USD    0
High    BTC-USD    0
Low     BTC-USD    0
Open    BTC-USD    0
Volume  BTC-USD    0
dtype: int64


In [5]:
btc_data

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2019-01-01,3843.520020,3850.913818,3707.231201,3746.713379,4324200990
2019-01-02,3943.409424,3947.981201,3817.409424,3849.216309,5244856836
2019-01-03,3836.741211,3935.685059,3826.222900,3931.048584,4530215219
2019-01-04,3857.717529,3865.934570,3783.853760,3832.040039,4847965467
2019-01-05,3845.194580,3904.903076,3836.900146,3851.973877,5137609824
...,...,...,...,...,...
2025-08-05,114141.445312,115117.437500,112701.109375,115072.187500,61039182286
2025-08-06,115028.000000,115737.835938,113372.250000,114140.914062,56379133510


In [6]:
from datetime import datetime, date

# Calculate the expected number of days
start_date = date(2019, 1, 1)
end_date = date(2025, 8, 9)
expected_days = (end_date - start_date).days + 1
print(f"Expected number of days: {expected_days}")

# Get the actual number of rows in the DataFrame
actual_rows = len(btc_data)
print(f"Actual number of rows: {actual_rows}")

# Compare the two numbers
if actual_rows == expected_days:
    print("All data points are available in the DataFrame.")
else:
    print(f"There are missing data points. Expected: {expected_days}, Actual: {actual_rows}")

Expected number of days: 2413
Actual number of rows: 2413
All data points are available in the DataFrame.


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [8]:
btc_data_clean = btc_data.copy()

In [9]:
# ============================================================================
# STEP 1: LOAD AND PREPARE DATA
# ============================================================================
print("=== STEP 1: DATA PREPARATION ===")

# Assuming you have btc_data_clean loaded
# Let's check the structure first
print("Data shape:", btc_data_clean.shape)
print("Columns:", btc_data_clean.columns.tolist())

# Extract the close price and date columns
# Adjust these column names based on your actual data structure

=== STEP 1: DATA PREPARATION ===
Data shape: (2413, 5)
Columns: [('Close', 'BTC-USD'), ('High', 'BTC-USD'), ('Low', 'BTC-USD'), ('Open', 'BTC-USD'), ('Volume', 'BTC-USD')]


In [10]:
btc_data_clean = btc_data_clean.reset_index()
print("DataFrame after resetting index:")
print(btc_data_clean.head())

DataFrame after resetting index:
Price        Date        Close         High          Low         Open  \
Ticker                 BTC-USD      BTC-USD      BTC-USD      BTC-USD   
0      2019-01-01  3843.520020  3850.913818  3707.231201  3746.713379   
1      2019-01-02  3943.409424  3947.981201  3817.409424  3849.216309   
2      2019-01-03  3836.741211  3935.685059  3826.222900  3931.048584   
3      2019-01-04  3857.717529  3865.934570  3783.853760  3832.040039   
4      2019-01-05  3845.194580  3904.903076  3836.900146  3851.973877   

Price       Volume  
Ticker     BTC-USD  
0       4324200990  
1       5244856836  
2       4530215219  
3       4847965467  
4       5137609824  


In [11]:
try:
    close_prices = btc_data_clean[('Close', 'BTC-USD')].astype(float)
    dates = pd.to_datetime(btc_data_clean[('Date', '')])
except:
    # Alternative column access if the above doesn't work
    close_prices = btc_data_clean['Close'].astype(float)
    dates = pd.to_datetime(btc_data_clean['Date'])

print(f"Date range: {dates.min()} to {dates.max()}")
print(f"Price range: ${close_prices.min():.2f} to ${close_prices.max():.2f}")


Date range: 2019-01-01 00:00:00 to 2025-08-09 00:00:00
Price range: $3399.47 to $119995.41


In [12]:
# ============================================================================
# STEP 2: CREATE PROPER FEATURES (NO LEAKAGE)
# ============================================================================
print("\n=== STEP 2: FEATURE ENGINEERING ===")

def create_features_no_leakage(prices, dates):
    """
    Create features that don't leak future information
    All features at time t use only data from times < t
    """
    df = pd.DataFrame({
        'Date': dates,
        'Close': prices
    }).reset_index(drop=True)

    # Sort by date to ensure proper time order
    df = df.sort_values('Date').reset_index(drop=True)

    # ===== LAG FEATURES (Past prices) =====
    df['Close_1day_ago'] = df['Close'].shift(1)
    df['Close_2days_ago'] = df['Close'].shift(2)
    df['Close_3days_ago'] = df['Close'].shift(3)
    df['Close_7days_ago'] = df['Close'].shift(7)

    # ===== MOVING AVERAGES (Using only past data) =====
    # Simple Moving Averages
    df['SMA_5'] = df['Close'].shift(1).rolling(window=5, min_periods=1).mean()
    df['SMA_10'] = df['Close'].shift(1).rolling(window=10, min_periods=1).mean()
    df['SMA_20'] = df['Close'].shift(1).rolling(window=20, min_periods=1).mean()
    df['SMA_30'] = df['Close'].shift(1).rolling(window=30, min_periods=1).mean()

    # Exponential Moving Averages
    df['EMA_5'] = df['Close'].shift(1).ewm(span=5).mean()
    df['EMA_20'] = df['Close'].shift(1).ewm(span=20).mean()
    # ===== PRICE RATIOS AND MOMENTUM =====
    df['Price_Change_1d'] = (df['Close_1day_ago'] - df['Close_2days_ago']) / df['Close_2days_ago']
    df['Price_Change_3d'] = (df['Close_1day_ago'] - df['Close_3days_ago']) / df['Close_3days_ago']
    df['Price_Change_7d'] = (df['Close_1day_ago'] - df['Close_7days_ago']) / df['Close_7days_ago']

    # Moving average ratios
    df['Close_to_SMA5_ratio'] = df['Close_1day_ago'] / df['SMA_5']
    df['Close_to_SMA20_ratio'] = df['Close_1day_ago'] / df['SMA_20']
    df['SMA5_to_SMA20_ratio'] = df['SMA_5'] / df['SMA_20']

    # ===== VOLATILITY FEATURES =====
    df['Rolling_Std_5'] = df['Close'].shift(1).rolling(window=5, min_periods=1).std()
    df['Rolling_Std_20'] = df['Close'].shift(1).rolling(window=20, min_periods=1).std()

    # ===== DATE FEATURES =====
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['DayOfYear'] = df['Date'].dt.dayofyear

    # ===== CYCLICAL FEATURES =====
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['DayOfWeek_sin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
    df['DayOfWeek_cos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)

    return df

# Create features
feature_df = create_features_no_leakage(close_prices, dates)
print("Features created successfully!")
print(f"Feature DataFrame shape: {feature_df.shape}")




=== STEP 2: FEATURE ENGINEERING ===
Features created successfully!
Feature DataFrame shape: (2413, 29)


In [13]:
# Remove rows with NaN values (first few rows will have NaN due to lag features)
clean_df = feature_df.dropna().reset_index(drop=True)
print(f"Clean data shape after removing NaN: {clean_df.shape}")

Clean data shape after removing NaN: (2406, 29)


In [14]:
# ============================================================================
# STEP 3: PREPARE TRAINING DATA
# ============================================================================
print("\n=== STEP 3: PREPARE TRAINING DATA ===")

# Define feature columns (everything except Date and Close)
feature_columns = [col for col in feature_df.columns if col not in ['Date', 'Close']]
print(f"Number of features: {len(feature_columns)}")
print("Features:", feature_columns)


=== STEP 3: PREPARE TRAINING DATA ===
Number of features: 27
Features: ['Close_1day_ago', 'Close_2days_ago', 'Close_3days_ago', 'Close_7days_ago', 'SMA_5', 'SMA_10', 'SMA_20', 'SMA_30', 'EMA_5', 'EMA_20', 'Price_Change_1d', 'Price_Change_3d', 'Price_Change_7d', 'Close_to_SMA5_ratio', 'Close_to_SMA20_ratio', 'SMA5_to_SMA20_ratio', 'Rolling_Std_5', 'Rolling_Std_20', 'Year', 'Month', 'Day', 'DayOfWeek', 'DayOfYear', 'Month_sin', 'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos']


In [15]:
# Remove rows with NaN values (first few rows will have NaN due to lag features)
clean_df = feature_df.dropna().reset_index(drop=True)
print(f"Clean data shape after removing NaN: {clean_df.shape}")

Clean data shape after removing NaN: (2406, 29)


In [16]:
X = clean_df[feature_columns]
y = clean_df['Close']

print(f"Training data: X shape {X.shape}, y shape {y.shape}")
print(f"Price range in clean data: ${y.min():.2f} - ${y.max():.2f}")

Training data: X shape (2406, 27), y shape (2406,)
Price range in clean data: $3399.47 - $119995.41


In [17]:
print("\n=== STEP 4: MODEL TRAINING ===")

# Use TimeSeriesSplit for proper time series validation
tscv = TimeSeriesSplit(n_splits=5)
splits = list(tscv.split(X))

# Use the last split for final training/testing
train_idx, test_idx = splits[-1]
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Test period: {clean_df.iloc[test_idx]['Date'].min()} to {clean_df.iloc[test_idx]['Date'].max()}")

# Train Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Train Random Forest (often better for financial data)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)



=== STEP 4: MODEL TRAINING ===
Training set: 2005 samples
Test set: 401 samples
Test period: 2024-07-05 00:00:00 to 2025-08-09 00:00:00


RandomForestRegressor(n_jobs=-1, random_state=42)

In [18]:
print("\n=== STEP 5: MODEL EVALUATION ===")

def evaluate_model(model, model_name, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    avg_price = y_test.mean()
    mae_pct = (mae / avg_price) * 100
    rmse_pct = (rmse / avg_price) * 100

    print(f"\n{model_name} Results:")
    print(f"MAE: {mae:.2f} (${mae:.2f}) - {mae_pct:.2f}% of avg price")
    print(f"RMSE: {rmse:.2f} (${rmse:.2f}) - {rmse_pct:.2f}% of avg price")
    print(f"R²: {r2:.4f}")

    return {'model': model, 'mae': mae, 'rmse': rmse, 'r2': r2, 'predictions': y_pred}

# Evaluate both models
lr_results = evaluate_model(lr_model, "Linear Regression", X_test, y_test)
rf_results = evaluate_model(rf_model, "Random Forest", X_test, y_test)

# Choose the best model based on RMSE
best_model_results = lr_results if lr_results['rmse'] < rf_results['rmse'] else rf_results
best_model_name = "Linear Regression" if lr_results['rmse'] < rf_results['rmse'] else "Random Forest"
best_model = best_model_results['model']

print(f"\nBest Model: {best_model_name}")

# ============================================================================


=== STEP 5: MODEL EVALUATION ===

Linear Regression Results:
MAE: 1511.51 ($1511.51) - 1.73% of avg price
RMSE: 2097.80 ($2097.80) - 2.40% of avg price
R²: 0.9878

Random Forest Results:
MAE: 21954.56 ($21954.56) - 25.12% of avg price
RMSE: 27381.64 ($27381.64) - 31.33% of avg price
R²: -1.0779

Best Model: Linear Regression


In [19]:
print("\n=== STEP 6: 7-DAY FUTURE PREDICTION ===")

def predict_future_prices(model, feature_df, feature_columns, n_days=7):
    """
    Predict future prices for n_days
    """
    # Get the last valid row of features
    last_row = feature_df.dropna().iloc[-1].copy()
    last_date = last_row['Date']

    # Keep track of recent prices for calculating new features
    recent_prices = feature_df['Close'].dropna().tolist()

    predictions = []

    for day in range(1, n_days + 1):
        # Calculate the future date
        future_date = last_date + timedelta(days=day)

        # Create new feature row
        new_row = last_row.copy()

        # Update date features
        new_row['Year'] = future_date.year
        new_row['Month'] = future_date.month
        new_row['Day'] = future_date.day
        new_row['DayOfWeek'] = future_date.weekday()
        new_row['DayOfYear'] = future_date.timetuple().tm_yday

        # Update cyclical features
        new_row['Month_sin'] = np.sin(2 * np.pi * new_row['Month'] / 12)
        new_row['Month_cos'] = np.cos(2 * np.pi * new_row['Month'] / 12)
        new_row['DayOfWeek_sin'] = np.sin(2 * np.pi * new_row['DayOfWeek'] / 7)
        new_row['DayOfWeek_cos'] = np.cos(2 * np.pi * new_row['DayOfWeek'] / 7)

        # Update lag features with recent prices
        if len(recent_prices) >= 1:
            new_row['Close_1day_ago'] = recent_prices[-1]
        if len(recent_prices) >= 2:
            new_row['Close_2days_ago'] = recent_prices[-2]
        if len(recent_prices) >= 3:
            new_row['Close_3days_ago'] = recent_prices[-3]
        if len(recent_prices) >= 7:
            new_row['Close_7days_ago'] = recent_prices[-7]

        # Update moving averages
        if len(recent_prices) >= 5:
            new_row['SMA_5'] = np.mean(recent_prices[-5:])
            new_row['EMA_5'] = recent_prices[-1]  # Simplified EMA
        if len(recent_prices) >= 10:
            new_row['SMA_10'] = np.mean(recent_prices[-10:])
        if len(recent_prices) >= 20:
            new_row['SMA_20'] = np.mean(recent_prices[-20:])
            new_row['EMA_20'] = recent_prices[-1]  # Simplified EMA
        if len(recent_prices) >= 30:
            new_row['SMA_30'] = np.mean(recent_prices[-30:])

        # Update price changes and ratios
        if len(recent_prices) >= 2:
            new_row['Price_Change_1d'] = (recent_prices[-1] - recent_prices[-2]) / recent_prices[-2]
        if len(recent_prices) >= 3:
            new_row['Price_Change_3d'] = (recent_prices[-1] - recent_prices[-3]) / recent_prices[-3]
        if len(recent_prices) >= 7:
            new_row['Price_Change_7d'] = (recent_prices[-1] - recent_prices[-7]) / recent_prices[-7]

        # Update ratios
        if 'SMA_5' in new_row and new_row['SMA_5'] > 0:
            new_row['Close_to_SMA5_ratio'] = recent_prices[-1] / new_row['SMA_5']
        if 'SMA_20' in new_row and new_row['SMA_20'] > 0:
            new_row['Close_to_SMA20_ratio'] = recent_prices[-1] / new_row['SMA_20']
        if 'SMA_5' in new_row and 'SMA_20' in new_row and new_row['SMA_20'] > 0:
            new_row['SMA5_to_SMA20_ratio'] = new_row['SMA_5'] / new_row['SMA_20']

        # Update volatility features
        if len(recent_prices) >= 5:
            new_row['Rolling_Std_5'] = np.std(recent_prices[-5:])
        if len(recent_prices) >= 20:
            new_row['Rolling_Std_20'] = np.std(recent_prices[-20:])

        # Make prediction
        feature_vector = new_row[feature_columns].values.reshape(1, -1)
        predicted_price = model.predict(feature_vector)[0]

        # Store prediction
        predictions.append({
            'Day': day,
            'Date': future_date.strftime('%Y-%m-%d'),
            'Predicted_Close': predicted_price
        })

        # Add prediction to recent prices for next iteration
        recent_prices.append(predicted_price)

        # Update last_row for next iteration
        last_row = new_row
        last_row['Close'] = predicted_price

    return pd.DataFrame(predictions)

# Generate 7-day predictions
future_predictions = predict_future_prices(best_model, feature_df, feature_columns, n_days=7)

print("7-Day Bitcoin Price Predictions:")
print(future_predictions.round(2))

# Calculate some statistics
current_price = clean_df['Close'].iloc[-1]
final_predicted_price = future_predictions['Predicted_Close'].iloc[-1]
total_change = ((final_predicted_price - current_price) / current_price) * 100

print(f"\nPrediction Summary:")
print(f"Current Price (last known): ${current_price:.2f}")
print(f"Day 7 Predicted Price: ${final_predicted_price:.2f}")
print(f"Total Predicted Change: {total_change:+.2f}%")
print(f"Prediction Range: ${future_predictions['Predicted_Close'].min():.2f} - ${future_predictions['Predicted_Close'].max():.2f}")

# ===================================


=== STEP 6: 7-DAY FUTURE PREDICTION ===
7-Day Bitcoin Price Predictions:
   Day        Date  Predicted_Close
0    1  2025-08-10        115730.60
1    2  2025-08-11        115500.68
2    3  2025-08-12        115356.71
3    4  2025-08-13        115158.22
4    5  2025-08-14        114909.34
5    6  2025-08-15        114830.27
6    7  2025-08-16        114782.80

Prediction Summary:
Current Price (last known): $116500.36
Day 7 Predicted Price: $114782.80
Total Predicted Change: -1.47%
Prediction Range: $114782.80 - $115730.60


In [20]:
# STEP 7: MODEL INTERPRETATION
# ============================================================================
print("\n=== STEP 7: FEATURE IMPORTANCE ===")

if best_model_name == "Random Forest":
    feature_importance = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    print("Top 10 Most Important Features:")
    print(feature_importance.head(10))
else:
    coefficients = pd.DataFrame({
        'Feature': feature_columns,
        'Coefficient': best_model.coef_
    }).sort_values('Coefficient', key=abs, ascending=False)

    print("Top 10 Features by Coefficient Magnitude:")
    print(coefficients.head(10))

print("\n=== PREDICTION COMPLETE ===")
print("Note: These predictions are based on historical patterns and should not be used as financial advice.")
print("Bitcoin prices are highly volatile and unpredictable!")


=== STEP 7: FEATURE IMPORTANCE ===
Top 10 Features by Coefficient Magnitude:
                 Feature  Coefficient
13   Close_to_SMA5_ratio -6608.477355
14  Close_to_SMA20_ratio  5533.224548
15   SMA5_to_SMA20_ratio -5054.310306
11       Price_Change_3d  1412.214611
10       Price_Change_1d   585.446488
19                 Month  -199.326681
18                  Year    59.468182
24             Month_cos    57.616225
26         DayOfWeek_cos    47.224116
12       Price_Change_7d    35.652016

=== PREDICTION COMPLETE ===
Note: These predictions are based on historical patterns and should not be used as financial advice.
Bitcoin prices are highly volatile and unpredictable!
